In [70]:
import pandas as pd
from torch.utils.data import DataLoader,Dataset, Subset
import numpy as np
import tft_model
from data_formatters import ts_dataset  
import data_formatters.base
import expt_settings.configs
import importlib
from data_formatters import utils
import torch.optim as optim


In [58]:
importlib.reload(tft_model)
importlib.reload(utils)

<module 'data_formatters.utils' from '/Users/matthewsherar/Developer/Temporal Fusion Transformer/data_formatters/utils.py'>

In [2]:
ExperimentConfig = expt_settings.configs.ExperimentConfig

config = ExperimentConfig('electricity', 'outputs')
data_formatter = config.make_data_formatter()


print("*** Training from defined parameters for {} ***".format('electricity'))
data_csv_path = 'data/hourly_electricity.csv'
print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration(
)

# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()

*** Training from defined parameters for electricity ***
Loading & splitting data...


/Users/matthewsherar/Developer/Temporal Fusion Transformer/pytorch/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Formatting train-valid-test splits.
Setting scalers with training data...


In [3]:
len(train.id.unique())

369

In [4]:
id_col = 'categorical_id'
time_col='hours_from_start'
input_cols =['power_usage', 'hour', 'day_of_week', 'hours_from_start', 'categorical_id']
target_col = 'power_usage'
time_steps=192
num_encoder_steps = 168
output_size = 1
max_samples = 5000
input_size = 5

elect = ts_dataset.TSDataset(id_col, time_col, input_cols,
                      target_col, time_steps, max_samples,
                     input_size, num_encoder_steps, output_size, train)

Getting valid sampling locations.
Extracting 5000 samples...


In [5]:
batch_size=8
loader = DataLoader(
            elect,
            batch_size=batch_size,
            num_workers=2,
            shuffle=True
        )

In [6]:
for batch in loader:
    break

In [44]:
static_cols = ['meter']
categorical_cols = ['hour']
real_cols = ['power_usage','hour', 'day']
config = {}
config['static_variables'] = len(static_cols)
config['time_varying_categoical_variables'] = 1
config['time_varying_real_variables'] = 4

config['static_embedding_vocab_sizes'] = [369]
config['time_varying_embedding_vocab_sizes'] = [369]
config['embedding_dim'] = 8
config['lstm_hidden_dimension'] = 64
config['lstm_layers'] = 2
config['dropout'] = 0.05
config['device'] = 'cpu'
config['batch_size'] = 8
config['encode_length'] = 168
config['attn_heads'] = 1
config['num_quantiles'] = 3
config['vailid_quantiles'] = [0.25,0.5,0.75]

In [81]:
model = tft_model.TFT(config)

In [50]:
output,encoder_output, decoder_output, attn, attn_output_weights = model.forward(batch)

In [51]:
output.shape

torch.Size([8, 24, 3])

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001)
model.train()
epochs=15
losses = []
for i in range(epochs):
    epoch_loss = [] 
    for batch in loader:
        output, encoder_ouput, decoder_output, attn, attn_weights = model(batch)
        q_loss = torch.zeros(3)
        for i, q in enumerate(model.valid_quantiles):
            q_loss[i] = utils.pytorch_quantile_loss(batch['outputs'][:,:,0].flatten(), output[:,:,i].flatten(), q)
        loss = q_loss.mean()
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
    losses.append(np.mean(epoch_loss))
    print(losses[i])
    